In [1]:
!pip install -qU crewai[tools,agentops]==0.114.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 6.4 MB/s eta 0:00:00
ERROR: Cannot install crewai, crewai-tools and crewai[agentops,tools]==0.114.0 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [2]:
!pip install -qU tavily-python scrapegraph-py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 2.3 MB/s eta 0:00:00


In [6]:
from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import tool
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource
import agentops
from google.colab import userdata
from pydantic import BaseModel, Field
from typing import List
from tavily import TavilyClient
from scrapegraph_py import Client

import os
import json

In [7]:
# please register Agentops account then go to
# (https://app.agentops.ai/get-started)
# to get the api key

os.environ["AGENTOPS_API_KEY"] = userdata.get('agentops-colab')

agentops.init(
    api_key=userdata.get('agentops-colab'),
    skip_auto_end_session=True,
    default_tags=['crewai']
)

In [8]:
print(agentops.get_client().config.exporter_endpoint)

https://otlp.agentops.ai/v1/traces


In [9]:
output_dir = "./ai-agent-output"
os.makedirs(output_dir, exist_ok=True)

OPENROUTER_API_KEY = userdata.get('OPENROUTER_API_KEY')
basic_llm = LLM(
    model="openai/gpt-4o-mini",
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY
)

search_client = TavilyClient(api_key=userdata.get('tvly-search'))
scrape_client = Client(api_key=userdata.get('scrapegraph'))

In [10]:
no_keywords = 10

about_company = "Rankyx is a company that provides AI solutions to help websites refine their search and recommendation systems."

company_context = StringKnowledgeSource(
    content=about_company
)

## Setup Agents

### Agent: A

In [11]:
class SuggestedSearchQueries(BaseModel):
    queries: List[str] = Field(..., title="Suggested search queries to be passed to the search engine",
                               min_items=1, max_items=no_keywords)

search_queries_recommendation_agent = Agent(
    role="Search Queries Recommendation Agent",
    goal="\n".join([
                "To provide a list of suggested search queries to be passed to the search engine.",
                "The queries must be varied and looking for specific items."
            ]),
    backstory="The agent is designed to help in looking for products by providing a list of suggested search queries to be passed to the search engine based on the context provided.",
    llm=basic_llm,
    verbose=True,
)

search_queries_recommendation_task = Task(
    description="\n".join([
        "Rankyx is looking to buy {product_name} at the best prices (value for a price strategy)",
        "The campany target any of these websites to buy from: {websites_list}",
        "The company wants to reach all available proucts on the internet to be compared later in another stage.",
        "The stores must sell the product in {country_name}",
        "Generate at maximum {no_keywords} queries.",
        "The search keywords must be in {language} language.",
        "Search keywords must contains specific brands, types or technologies. Avoid general keywords.",
        "The search query must reach an ecommerce webpage for product, and not a blog or listing page."
    ]),
    expected_output="A JSON object containing a list of suggested search queries.",
    output_json=SuggestedSearchQueries,
    output_file=os.path.join(output_dir, "step_1_suggested_search_queries.json"),
    agent=search_queries_recommendation_agent
)

### Agent: B

In [12]:
class SignleSearchResult(BaseModel):
    title: str
    url: str = Field(..., title="the page url")
    content: str
    score: float
    search_query: str

class AllSearchResults(BaseModel):
    results: List[SignleSearchResult]

@tool
def search_engine_tool(query: str):
    """Useful for search-based queries. Use this to find current information about any query related pages using a search engine"""
    return search_client.search(query)

search_engine_agent = Agent(
    role="Search Engine Agent",
    goal="To search for products based on the suggested search query",
    backstory="The agent is designed to help in looking for products by searching for products based on the suggested search queries.",
    llm=basic_llm,
    verbose=True,
    tools=[search_engine_tool]
)

search_engine_task = Task(
    description="\n".join([
        "The task is to search for products based on the suggested search queries.",
        "You have to collect results from multiple search queries.",
        "Ignore any susbicious links or not an ecommerce single product website link.",
        "Ignore any search results with confidence score less than ({score_th}) .",
        "The search results will be used to compare prices of products from different websites.",
    ]),
    expected_output="A JSON object containing the search results.",
    output_json=AllSearchResults,
    output_file=os.path.join(output_dir, "step_2_search_results.json"),
    agent=search_engine_agent
)

### Agent: C

In [13]:
class ProductSpec(BaseModel):
    specification_name: str
    specification_value: str

class SingleExtractedProduct(BaseModel):
    page_url: str = Field(..., title="The original url of the product page")
    product_title: str = Field(..., title="The title of the product")
    product_image_url: str = Field(..., title="The url of the product image")
    product_url: str = Field(..., title="The url of the product")
    product_current_price: float = Field(..., title="The current price of the product")
    product_original_price: float = Field(title="The original price of the product before discount. Set to None if no discount", default=None)
    product_discount_percentage: float = Field(title="The discount percentage of the product. Set to None if no discount", default=None)

    product_specs: List[ProductSpec] = Field(..., title="The specifications of the product. Focus on the most important specs to compare.", min_items=1, max_items=5)

    agent_recommendation_rank: int = Field(..., title="The rank of the product to be considered in the final procurement report. (out of 5, Higher is Better) in the recommendation list ordering from the best to the worst")
    agent_recommendation_notes: List[str]  = Field(..., title="A set of notes why would you recommend or not recommend this product to the company, compared to other products.")


class AllExtractedProducts(BaseModel):
    products: List[SingleExtractedProduct]


@tool
def web_scraping_tool(page_url: str):
    """
    An AI Tool to help an agent to scrape a web page

    Example:
    web_scraping_tool(
        page_url="https://www.noon.com/egypt-en/15-bar-fully-automatic-espresso-machine-1-8-l-1500"
    )
    """
    details = scrape_client.smartscraper(
        website_url=page_url,
        user_prompt = "Extract ```json\n" + json.dumps(SingleExtractedProduct.model_json_schema()) + "```\n From the web page"

    )

    return {
        "page_url": page_url,
        "details": details
    }

scraping_agent = Agent(
    role="Web scraping agent",
    goal="To extract details from any website",
    backstory="The agent is designed to help in looking for required values from any website url. These details will be used to decide which best product to buy.",
    llm=basic_llm,
    tools=[web_scraping_tool],
    verbose=True,
)

scraping_task = Task(
    description="\n".join([
        "The task is to extract product details from any ecommerce store page url.",
        "The task has to collect results from multiple pages urls.",
        "Collect the best {top_recommendations_no} products from the search results.",
    ]),
    expected_output="A JSON object containing products details",
    output_json=AllExtractedProducts,
    output_file=os.path.join(output_dir, "step_3_search_results.json"),
    agent=scraping_agent
)

### Agent: D

In [14]:
procurement_report_author_agent = Agent(
    role="Procurement Report Author Agent",
    goal="To generate a professional HTML page for the procurement report",
    backstory="The agent is designed to assist in generating a professional HTML page for the procurement report after looking into a list of products.",
    llm=basic_llm,
    verbose=True,
)

procurement_report_author_task = Task(
    description="\n".join([
        "The task is to generate a professional HTML page for the procurement report.",
        "You have to use Bootstrap CSS framework for a better UI.",
        "Use the provided context about the company to make a specialized report.",
        "The report will include the search results and prices of products from different websites.",
        "The report should be structured with the following sections:",
        "1. Executive Summary: A brief overview of the procurement process and key findings.",
        "2. Introduction: An introduction to the purpose and scope of the report.",
        "3. Methodology: A description of the methods used to gather and compare prices.",
        "4. Findings: Detailed comparison of prices from different websites, including tables and charts.",
        "5. Analysis: An analysis of the findings, highlighting any significant trends or observations.",
        "6. Recommendations: Suggestions for procurement based on the analysis.",
        "7. Conclusion: A summary of the report and final thoughts.",
        "8. Appendices: Any additional information, such as raw data or supplementary materials.",
    ]),

    expected_output="A professional HTML page for the procurement report.",
    output_file=os.path.join(output_dir, "step_4_procurement_report.html"),
    agent=procurement_report_author_agent,
)

## Run the AI Crew

In [15]:
rankyx_crew = Crew(
    agents=[
        search_queries_recommendation_agent,
        search_engine_agent,
        scraping_agent,
        procurement_report_author_agent,
    ],
    tasks=[
        search_queries_recommendation_task,
        search_engine_task,
        scraping_task,
        procurement_report_author_task,
    ],
    process=Process.sequential,
    knowledge_sources=[company_context]
)

In [16]:
crew_results = rankyx_crew.kickoff(
    inputs={
        "product_name": "coffee machine for the office",
        "websites_list": ["www.amazon.eg", "www.jumia.com.eg", "www.noon.com/egypt-en"],
        "country_name": "Egypt",
        "no_keywords": 10,
        "language": "English",
        "score_th": 0.10,
        "top_recommendations_no": 10
    }
)

# Agent: Search Queries Recommendation Agent
## Task: Rankyx is looking to buy coffee machine for the office at the best prices (value for a price strategy)
The campany target any of these websites to buy from: ['www.amazon.eg', 'www.jumia.com.eg', 'www.noon.com/egypt-en']
The company wants to reach all available proucts on the internet to be compared later in another stage.
The stores must sell the product in Egypt
Generate at maximum 10 queries.
The search keywords must be in English language.
Search keywords must contains specific brands, types or technologies. Avoid general keywords.
The search query must reach an ecommerce webpage for product, and not a blog or listing page.


# Agent: Search Queries Recommendation Agent
## Final Answer: 
{
  "queries": [
    "Nespresso coffee machine site:www.amazon.eg",
    "DeLonghi espresso machine site:www.jumia.com.eg",
    "Breville coffee maker site:www.noon.com/egypt-en",
    "Philips 2200 Series coffee machine site:www.amazon.eg",
    "K

🖇 AgentOps: ToolEvent() is deprecated and will be removed in v4 in the future. Automatically tracked in v4.




# Agent: Search Engine Agent
## Thought: Action: search_engine_tool
## Using tool: search_engine_tool
## Tool Input: 
"{\"query\": \"Nespresso coffee machine site:www.amazon.eg\"}"
## Tool Output: 
{'query': 'Nespresso coffee machine', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.amazon.eg/-/en/Nespresso-Essenza-Coffee-Machine-Metal/dp/B098TYH53P', 'title': 'Nespresso Essenza Mini Coffee Machine, Metal', 'content': 'Buy Nespresso Essenza Mini Coffee Machine, Metal online on Amazon.eg at best prices. ✓ Fast and Free Shipping ✓ Free Returns ✓Cash on Delivery available on', 'score': 0.99992514, 'raw_content': None}, {'url': 'https://www.amazon.eg/-/en/Coffee-Machines-Nespresso-Tea-Espresso/s?rh=n%3A21864088031%2Cp_89%3ANespresso', 'title': 'Nespresso - Coffee Machines / Coffee, Tea & Espresso ...', 'content': "* Your amazon.eg Price, product pageEGP\xa06,499.00EGP6,499.00 List: EGP\xa012,999.00 Price, product pageEGP\xa06,999.00EGP6,999.00 

/usr/local/lib/python3.12/dist-packages/agentops/instrumentation/__init__.py:384: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  return _original_builtins_import(name, globals_dict, locals_dict, fromlist, level)



Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.I

🖇 AgentOps: ErrorEvent() is deprecated and will be removed in v4 in the future. Automatically tracked in v4.


 

I encountered an error while trying to use the tool. This was the error: [402] Insufficient credits.
 Tool web_scraping_tool accepts these inputs: Tool Name: web_scraping_tool
Tool Arguments: {'page_url': {'description': None, 'type': 'str'}}
Tool Description: 
    An AI Tool to help an agent to scrape a web page

    Example:
    web_scraping_tool(
        page_url="https://www.noon.com/egypt-en/15-bar-fully-automatic-espresso-machine-1-8-l-1500"
    )
    



# Agent: Web scraping agent
## Thought: I will start by scraping the necessary product details from the provided ecommerce websites. I'll begin with the first query about the Nespresso coffee machine from Amazon Egypt.
## Using tool: web_scraping_tool
## Tool Input: 
"{\"page_url\": \"https://www.amazon.eg/-/en/Nespresso-Essenza-Coffee-Machine-Metal/dp/B098TYH53P\"}"
## Tool Output: 

I encountered an error while trying to use the tool. This was the error: [402] Insufficient credits.
 Tool web_scraping_tool accepts these inpu